In [1]:
import os
from os.path import join, dirname
from dotenv import load_dotenv

from langchain_groq import ChatGroq

load_dotenv()
GROQ_API_KEY = os.environ.get("GROQ_CLOUD")

In [2]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = GROQ_API_KEY,
    model_name = "llama-3.1-70b-versatile"
)

response = llm.invoke("What's the number 1 tip to become a great software engineer?")
print(response.content)

The number 1 tip to become a great software engineer is to **"Write Code Every Day"**. Consistency is key to improving your coding skills. Here's why:

1. **Practice reinforces learning**: The more you code, the more you'll retain what you've learned. It's like playing a musical instrument - the more you practice, the better you'll become.
2. **Develops muscle memory**: Writing code regularly helps develop muscle memory, making it easier to recall syntax, patterns, and best practices.
3. **Improves problem-solving skills**: Coding every day exposes you to different problems and challenges, helping you develop your problem-solving skills and think critically.
4. **Enhances creativity**: The more you code, the more you'll be able to think creatively and come up with innovative solutions to complex problems.
5. **Builds a portfolio**: Writing code every day allows you to build a portfolio of projects, demonstrating your skills to potential employers or clients.

To make the most of this t

In [9]:
from langchain_community.document_loaders import WebBaseLoader

# loader = WebBaseLoader("https://jobs.nike.com/job/R-39879")
loader = WebBaseLoader("https://jobs.nike.com/job/R-39879")
page_data = loader.load().pop().page_content

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    # SCRAPPED TEXT FROM WEBSITE:
    {page_data}
    # INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input = {'page_data': page_data})
res.content

'```json\n{\n  "role": "Machine Learning Engineer - Supply Chain",\n  "experience": "1-3 years of professional experience in software engineering, data engineering, machine learning, or related field",\n  "skills": [\n    "Python",\n    "algorithms and data structures",\n    "AWS",\n    "database technology (e.g. Postgres, Redis)",\n    "data processing technology (e.g. EMR)",\n    "agile development",\n    "test driven development",\n    "MLOps",\n    "API development",\n    "mathematical optimization",\n    "cloud architecture",\n    "Amazon Web Services",\n    "Spark",\n    "Kubernetes",\n    "Docker",\n    "Jenkins",\n    "Databricks",\n    "Terraform"\n  ],\n  "description": "Develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact. Contribut

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Machine Learning Engineer - Supply Chain',
 'experience': '1-3 years of professional experience in software engineering, data engineering, machine learning, or related field',
 'skills': ['Python',
  'algorithms and data structures',
  'AWS',
  'database technology (e.g. Postgres, Redis)',
  'data processing technology (e.g. EMR)',
  'agile development',
  'test driven development',
  'MLOps',
  'API development',
  'mathematical optimization',
  'cloud architecture',
  'Amazon Web Services',
  'Spark',
  'Kubernetes',
  'Docker',
  'Jenkins',
  'Databricks',
  'Terraform'],
 'description': 'Develop robust advanced analytics and machine learning solutions that have a direct impact on the Supply Chain infrastructure. Design and implement scalable applications that leverage prediction models and optimization programs to deliver data driven decisions that result in immense business impact. Contribute to core advanced analytics and machine learning platforms and tools to enable b

In [14]:
import pandas as pd
df = pd.read_csv("sample_data.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="sample_data")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents= row["Techstack"],
            metadatas = {"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

In [22]:
links = collection.query(query_texts = json_res["skills"],  n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https

In [23]:
json_res["skills"]

['Python',
 'algorithms and data structures',
 'AWS',
 'database technology (e.g. Postgres, Redis)',
 'data processing technology (e.g. EMR)',
 'agile development',
 'test driven development',
 'MLOps',
 'API development',
 'mathematical optimization',
 'cloud architecture',
 'Amazon Web Services',
 'Spark',
 'Kubernetes',
 'Docker',
 'Jenkins',
 'Databricks',
 'Terraform']

In [24]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    You are Neo, a talented Machine Learning and software engineer dedicate to seamliess integration
    of business process with A.I and machine learning tools to drive revenue and reduce cost.
    Over our experience, we have empowered numeroud enterprises with tailored solutions fostering,
    scalability, process optimization, cost reduction and heightened overral efficiency.
    Your job is to write a cold email to potention employeers and clients regarding the job mentioned above
    describing you capabilities in fulfilling their needs.
    Also add the most work from the following links to showcase your abilities: {link_lst}
    Remember you are Neo a talented Machine Learning and Software Engineer.
    Do not provide preamble.
    ### EMAIL (NO PREAMBLE):

    """
)

chain_email = prompt_email | llm

res = chain_email.invoke({"job_description": str(json_res), "link_lst": links})
print(res.content)

Subject: Unlock the Power of AI and Machine Learning for Your Business

Dear [Potential Employer/Client],

I'm Neo, a seasoned Machine Learning and Software Engineer with a proven track record of seamlessly integrating business processes with AI and machine learning tools to drive revenue and reduce costs. With my expertise, I've empowered numerous enterprises to achieve scalability, process optimization, cost reduction, and heightened overall efficiency.

My capabilities include:

* Developing and deploying machine learning models using Python, TensorFlow, and scikit-learn
* Designing and implementing software solutions using Python, Java, and C++
* Integrating AI and machine learning tools with business processes to drive automation and efficiency
* Collaborating with cross-functional teams to identify business needs and develop tailored solutions
* Ensuring scalability, reliability, and maintainability of software systems

Some of my notable work includes:

* **Machine Learning and 